In [2]:
import numpy as np
import matplotlib.pyplot as plt

import librosa
import librosa.display

import tensorflow as tf
from tensorflow import keras
from tensorflow.compat import v1
from functools import partial

In [3]:
data = np.load('cqt.npz')
train = librosa.amplitude_to_db(data['spec'], ref=np.max).reshape(-1, 168,87,1) # cqt
train = (train+80)/80
instr_target = data['instr'][:,0] # instr
note_target = data['instr'][:,1] # note
print(train.shape, note_target.shape)
print(train.min(),train.max())

FileNotFoundError: [Errno 2] No such file or directory: 'cqt.npz'

In [ ]:
from sklearn.model_selection import train_test_split

train_data, test_data, train_target, test_target = train_test_split(
    train, target, test_size=0.1, stratify=target
)

print(train_data.shape, test_data.shape)

(18439, 168, 87, 1) (2049, 168, 87, 1)


In [ ]:
def U_net_model(input_tensor):
# U-net모델 만들기
## tools
    conv_filters = [16, 32, 64, 128, 256, 512]
    cont_factory = partial(
        keras.layers.Cont, kernel_size=(3,3), strides=1, padding="same"
    )
    cont_activation = keras.layers.ELU()
    
    expan_factory = partial(
        keras.layers.Conv2DTranspose,
        kernel_size=(3,3), strides=1, padding="same"
    )
    expan_activation = keras.layers.LeakyReLU(0.2)

## Contracting path
    ## 1
    conv1 = cont_factory(conv_filters[0])(input_tensor)
    batch1 = keras.layers.BatchNormalization(axis=-1)(conv1)
    rel1 = cont_activation(batch1)
    ## 2
    conv2 = cont_factory(conv_filters[1])(rel1)
    batch2 = keras.layers.BatchNormalization(axis=-1)(conv2)
    rel2 = cont_activation(batch2)
    ## 3
    conv3 = cont_factory(conv_filters[2])(rel2)
    batch3 = keras.layers.BatchNormalization(axis=-1)(conv3)
    rel3 = cont_activation(batch3)
    ## 4
    conv4 = cont_factory(conv_filters[3])(rel3)
    batch4 = keras.layers.BatchNormalization(axis=-1)(conv4)
    rel4 = cont_activation(batch4)
    ## 5
    conv5 = cont_factory(conv_filters[4])(rel4)
    batch5 = keras.layers.BatchNormalization(axis=-1)(conv5)
    rel5 = cont_activation(batch5)
    ## 6
    conv6 = cont_factory(conv_filters[5])(rel5)

## Expansive path
    ## 6
    up1 = expan_factory(conv_filters[4])(conv6)
    up1 = expan_activation(up1)
    up_batch1 = keras.layers.BatchNormalization(axis=-1)(up1)
    drop1 = keras.layers.Dropout(0.5)(up_batch1)
    merge1 = keras.layers.Concatenate(axis=-1)([conv5,drop1])
    ## 5
    up2 = expan_factory(conv_filters[3])(merge1)
    up2 = expan_activation(up2)
    up_batch2 = keras.layers.BatchNormalization(axis=-1)(up2)
    drop2 = keras.layers.Dropout(0.5)(up_batch2)
    merge2 = keras.layers.Concatenate(axis=-1)([conv4,drop2])
    ## 4
    up3 = expan_factory(conv_filters[2])(merge2)
    up3 = expan_activation(up3)
    up_batch3 = keras.layers.BatchNormalization(axis=-1)(up3)
    drop3 = keras.layers.Dropout(0.5)(up_batch3)
    merge3 = keras.layers.Concatenate(axis=-1)([conv3,drop3])
    ## 3
    up4 = expan_factory(conv_filters[1])(merge3)
    up4 = expan_activation(up4)
    up_batch4 = keras.layers.BatchNormalization(axis=-1)(up4)
    drop4 = keras.layers.Dropout(0.5)(up_batch4)
    merge4 = keras.layers.Concatenate(axis=-1)([conv2,drop4])
    ## 2
    up5 = expan_factory(conv_filters[0])(merge4)
    up5 = expan_activation(up5)
    up_batch5 = keras.layers.BatchNormalization(axis=-1)(up5)
    drop5 = keras.layers.Dropout(0.5)(up_batch5)
    merge5 = keras.layers.Concatenate(axis=-1)([conv1,drop5])
    ## 1
    up6 = expan_factory(1)(merge5)
    up6 = expan_activation(up6)
    up_batch6 = keras.layers.BatchNormalization(axis=-1)(up6)
## output
    return keras.layers.Conv2D(2, (4,4), dilation_rate=(2,2), activation="sigmoid", padding="same")(up_batch6)
    